**Data prep Notebook**

load data

In [7]:
import pandas as pd
xls = pd.ExcelFile("C:/Users/hp/Documents/Python_projects/internship project/ExtractionStagiaire 11.07.2025.xlsx")
sheet_names = xls.sheet_names

ADHERENT = xls.parse("ADHERENT")
AIDE_FRONTALE = xls.parse("AIDE_FRONTALE") 
AMC_PRESTATION = xls.parse("AMC_PRESTATIONS2011")
AMC_SINISTRES = xls.parse("AMC_SINISTRES2011")
ATS_2024 = xls.parse("ATS_2024")
Bourse = xls.parse("Bourse")
DATA_Zephyr = xls.parse("DATA__Zephyr_2021")
Estivage = xls.parse("Estivage")
FDS = xls.parse("FDS")
Nafida_INTERNET = xls.parse("Nafida_INTERNET")
Nafida_PC = xls.parse("Nafida_PC")
PELERINAGE = xls.parse("PELERINAGE")
PR_ONCF_TRAFIC = xls.parse("PR_ONCF_TRAFIC")
PR_SUPRATOURS_TRAFIC = xls.parse("PR_SUPRATOURS_TRAFIC")
PRESCOLAIRE = xls.parse("PRESCOLAIRE")
PROJET_PILOTE = xls.parse("PROJET_PILOTE")
YASSIR = xls.parse("YASSIR")



In [ ]:
bourse = bourse.drop_duplicates()
bourse["Montant"] = bourse["Montant"].fillna(bourse["Montant"].median())
bourse["Statut"] = bourse["Statut"].fillna("Inconnu")
bourse["Date"] = pd.to_datetime(bourse["Date"], errors="coerce")
bourse = bourse.dropna(subset=["CIN"])

Define alias for standardization 

In [32]:
cin_alias = ['CIN', 'CIN_BOURSIER']
date_alias = ["Date d'arrivée", "Date", "DateAccord", "Date de création", "date_comande", "DateCreation" ]
montant_alias = ["Montant", "Montant_Gere", "montant_crédit"]
status_alias = ['Statut_gestion', 'Statut paiment', 'LL_STATUT', 'STATUT']
Adherent_alias = ['NumeroAdherent', 'Num_Adherent_Adherent', 'N°_Adherent_Adherent', 'Num_Adherent', 'NUM_ADHERENT']


Container for cleaned data 

In [ ]:
standardized_table = []

for name in sheet_names :
    if name == "ADHERENT" :
        continue
    df = xls.parse(name)
    df_copy = df.copy()
    
    #Adhrent
    for alias in Adherent_alias :
        if alias in df_copy.columns :
            df_copy = df_copy.rename(columns={alias: 'Num Adherent'})
            break
    #rename CIN 
    for alias in cin_alias :
        if alias in df_copy.columns :
            df_copy = df_copy.rename(columns={alias: 'CIN'})
            break
    #rename date
    for alias in date_alias :
        if alias in df_copy.columns :
            df_copy = df_copy.rename(columns={alias: 'Date'})
            break
    #rename montant
    for alias in montant_alias :
        if alias in df_copy.columns :
            df_copy = df_copy.rename(columns={alias: 'Montant'})
            break
    #rename status
    for alias in status_alias :
        if alias in df_copy.columns :
            df_copy = df_copy.rename(columns={alias: 'Statut'})
            break
        
    cols_to_keep = ['Num Adherent', 'CIN', 'Date', 'Montant', 'Statut']
    available_cols = [col for col in cols_to_keep if col in df_copy.columns]
    df_clean = df_copy[available_cols].copy()

    df_clean['Type prestation'] = name
    
    standardized_table.append(df_clean)
    
    
merged_df = pd.concat(standardized_table, ignore_index=True)
mean = round(merged_df['Montant'].mean(), 2)
merged_df['Montant'] = merged_df['Montant'].fillna(mean)
merged_df['Num Adherent'] = merged_df['Num Adherent'].fillna('---')
merged_df['CIN'] = merged_df['CIN'].fillna('---')
merged_df['Statut'] = merged_df['Statut'].fillna('---')
merged_df['Date'] = pd.to_datetime(merged_df['Date'], errors='coerce')
merged_df['Date'] = merged_df['Date'].fillna(merged_df['Date'].min())
merged_df['Date'] = merged_df['Date'].dt.date
print(merged_df.tail(10))

    Num Adherent  CIN   Montant Type prestation    Statut        Date
180   291925/000  ---  74581.82          YASSIR  Débloqué  2024-05-16
181   408887/000  ---  74581.82          YASSIR   Demandé  2025-06-06
182   215500/000  ---  74581.82          YASSIR  Débloqué  2022-10-17
183   518983/000  ---  74581.82          YASSIR  Débloqué  2024-06-25
184   389634/000  ---  74581.82          YASSIR  Débloqué  2023-07-18
185   547694/000  ---  74581.82          YASSIR  Débloqué  2023-05-10
186    99345/000  ---  74581.82          YASSIR  Débloqué  2025-04-23
187   425584/000  ---  74581.82          YASSIR  Débloqué  2024-12-13
188   539215/000  ---  74581.82          YASSIR  Débloqué  2024-04-17
189   346596/000  ---  74581.82          YASSIR  Débloqué  2023-12-18


save to CSV 

In [42]:
merged_df.to_excel('merged_tables.xlsx', index=False)